# PyTorchとGPUを使った手書き数字認識
Created: 2025.4.24 阿形航平

In [ ]:
# モジュールのインポート
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import time

In [ ]:
# CUDA (GPU)が利用できるか確認

is_cuda_available = torch.cuda.is_available()
print(f"is_cuda_available: {is_cuda_available}")

In [ ]:
# データローダの準備

# transformの用意
transform = transforms.ToTensor()

# 訓練データ
train_dataset = torchvision.datasets.MNIST(root='./data',
                                           train=True,
                                           transform=transform,
                                           download = True)
# テストデータ
test_dataset = torchvision.datasets.MNIST(root='./data',
                                           train=False,
                                           transform=transform,
                                           download = True)

# Batch読み込みの準備
batch_size = 256
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

In [ ]:
# ネットワークの定義
class MNISTModel1(nn.Module):
    def __init__(self, input_size, hidden1_size, hidden2_size, output_size):
        super().__init__()
        self.input_size = input_size
        
        self.fc1 = nn.Linear(input_size, hidden1_size)
        self.fc2 = nn.Linear(hidden1_size, hidden2_size) # 全結合層
        self.fc3 = nn.Linear(hidden2_size, output_size) # 出力層

    def forward(self, x: torch.Tensor):
        tmp = x.view(-1, self.input_size)
        tmp = self.fc1(tmp)
        tmp = F.relu(tmp)
        tmp = self.fc2(tmp)
        tmp = F.relu(tmp)
        tmp = self.fc3(tmp)
        y = F.log_softmax(tmp, dim=1)
        return y

In [ ]:
# 学習のための準備

# GPUデバイスの取得
device = torch.device('cuda:0' if is_cuda_available else 'cpu')

# モデルのインスタンス化
model = MNISTModel1(28*28, 512, 256, 10).to(device)

# 
criterion = nn.NLLLoss()

In [ ]:
# 学習開始

model.train()

# 最適化アルゴリズムを定義
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)

start_time = time.time()
for epoch in range(10):
    total_loss = 0
    for images, labels in train_loader:
        # GPUへの転送
        images, labels = images.to(device), labels.to(device)
        
        # 1. 勾配リセット
        optimizer.zero_grad()

        # 2. 推論
        outputs = model(images)

        # 3. 誤差計算
        loss = criterion(outputs, labels)

        # 4. 誤差逆伝播
        loss.backward()

        # 5. パラメータ更新
        optimizer.step()
        
        total_loss += loss.item()
    
    print(f'Epoch: {epoch + 1}, Loss: {total_loss / len(train_loader)}')
end_time = time.time()
duration = end_time - start_time
print(f"Finished training in {duration:.2f}s")

# Torch Script形式で保存
model_scripted = torch.jit.script(model)
model_scripted.save('model_scripted.pth')

In [ ]:
# Torch Script形式で読み込み
model = torch.jit.load('model_scripted.pth')

# test
correct = 0
total = 0
model.eval()
with torch.no_grad(): # 勾配を計算なし
    for images, labels in test_loader:
        # GPUへの転送
        images, labels = images.to(device), labels.to(device)
        
        # 
        outputs = model(images)

        # 
        _, predicted = torch.max(outputs.data, 1)
        
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
accuracy = correct / total
print(f"{total=}, {correct=}, {accuracy=}")